# Reddit Marketing AI Agent - Complete Example Workflow

This notebook demonstrates the complete workflow of the Reddit Marketing AI Agent, from document ingestion to response execution.

## Features Demonstrated:
1. **Setup & Configuration** - Environment validation and service initialization
2. **Organization Setup** - Create and configure an organization
3. **Document Ingestion** - Multiple methods (direct content, file upload, URL scraping)
4. **Campaign Creation** - Create and configure a marketing campaign
5. **Topic Discovery** - Extract relevant topics from documents
6. **Subreddit Discovery** - Find relevant subreddits based on topics
7. **Post Discovery** - Find relevant posts in target subreddits
8. **Response Generation** - AI-generated contextual responses
9. **Response Execution** - Post responses to Reddit (with safety controls)
10. **Analytics & Reporting** - Comprehensive performance analysis

## Safety Features:
- **Reddit Posting Control**: `ACTUALLY_POST_TO_REDDIT = False` prevents accidental posting
- **Credential Validation**: Checks for required API keys
- **Error Handling**: Graceful handling of API failures
- **Independent Cells**: Each step can be run independently

## 1. Setup & Configuration

In [ ]:
import requests
import json
import os
from typing import Dict, Any, List
from datetime import datetime
import time

# Configuration
API_BASE_URL = "http://localhost:8000/api/v1"
ORGANIZATION_ID = "example-org-2024"
ORGANIZATION_NAME = "Example Organization"

# Safety control - Set to True only when you want to actually post to Reddit
ACTUALLY_POST_TO_REDDIT = False

# Reddit credentials (replace with your actual credentials)
REDDIT_CREDENTIALS = {
    "client_id": "your_reddit_client_id",
    "client_secret": "your_reddit_client_secret",
    "username": "your_reddit_username",
    "password": "your_reddit_password"
}

print("🚀 Reddit Marketing AI Agent - Example Workflow")
print(f"📅 Started at: {datetime.now()}")
print(f"🌐 API Base URL: {API_BASE_URL}")
print(f"🏢 Organization: {ORGANIZATION_NAME} ({ORGANIZATION_ID})")
print(f"⚠️  Reddit Posting: {'ENABLED' if ACTUALLY_POST_TO_REDDIT else 'DISABLED (Safe Mode)'}")
print("\n" + "="*60)

In [ ]:
# Helper functions
def make_request(method: str, endpoint: str, data: Dict = None, params: Dict = None) -> Dict[str, Any]:
    """Make API request with error handling."""
    url = f"{API_BASE_URL}{endpoint}"
    
    try:
        if method.upper() == "GET":
            response = requests.get(url, params=params)
        elif method.upper() == "POST":
            response = requests.post(url, json=data, params=params)
        elif method.upper() == "DELETE":
            response = requests.delete(url, params=params)
        else:
            raise ValueError(f"Unsupported method: {method}")
        
        response.raise_for_status()
        return response.json()
    except requests.exceptions.RequestException as e:
        print(f"❌ API Error: {e}")
        if hasattr(e.response, 'text'):
            print(f"Response: {e.response.text}")
        return {"error": str(e)}

def print_response(title: str, response: Dict[str, Any]):
    """Pretty print API response."""
    print(f"\n📋 {title}")
    print("-" * 40)
    
    if "error" in response:
        print(f"❌ Error: {response['error']}")
        return
    
    if "success" in response:
        status = "✅" if response["success"] else "❌"
        print(f"{status} Status: {response.get('message', 'No message')}")
    
    if "data" in response and response["data"]:
        print(f"📊 Data: {json.dumps(response['data'], indent=2)}")

print("✅ Helper functions loaded")

In [ ]:
# Check API health
health_response = make_request("GET", "/health/")
print_response("API Health Check", health_response)

# Check detailed health
detailed_health = make_request("GET", "/health/detailed")
print_response("Detailed Health Check", detailed_health)

## 2. Organization Setup

In [ ]:
# List existing organizations
orgs_response = make_request("GET", "/documents/organizations")
print_response("Existing Organizations", orgs_response)

# Check if our organization exists
org_exists = False
if "data" in orgs_response and "organizations" in orgs_response["data"]:
    for org in orgs_response["data"]["organizations"]:
        if org["id"] == ORGANIZATION_ID:
            org_exists = True
            print(f"\n✅ Organization '{ORGANIZATION_NAME}' already exists")
            break

if not org_exists:
    print(f"\n📝 Organization '{ORGANIZATION_NAME}' will be created during document ingestion")

## 3. Document Ingestion

We'll demonstrate all three document ingestion methods:
1. **Direct Content Input** - Paste content directly
2. **Simulated File Upload** - Simulate uploading a file
3. **URL Scraping** - Scrape content from a URL

In [ ]:
# Method 1: Direct Content Input
print("📄 Method 1: Direct Content Input")

direct_documents = [
    {
        "title": "Python Best Practices Guide",
        "content": """
        Python Best Practices for Clean Code
        
        Writing clean, maintainable Python code is essential for any developer. Here are some key best practices:
        
        1. Follow PEP 8 Style Guide
        - Use 4 spaces for indentation
        - Keep lines under 79 characters
        - Use descriptive variable names
        
        2. Write Docstrings
        - Document all functions and classes
        - Use triple quotes for docstrings
        - Follow Google or NumPy docstring conventions
        
        3. Use Type Hints
        - Add type hints to function parameters and return values
        - Use typing module for complex types
        - Helps with IDE support and code documentation
        
        4. Error Handling
        - Use specific exception types
        - Handle exceptions gracefully
        - Log errors appropriately
        
        5. Testing
        - Write unit tests for all functions
        - Use pytest for testing framework
        - Aim for high test coverage
        
        These practices will help you write more maintainable and professional Python code.
        """,
        "metadata": {
            "category": "programming",
            "language": "python",
            "difficulty": "intermediate"
        }
    },
    {
        "title": "Machine Learning Fundamentals",
        "content": """
        Introduction to Machine Learning
        
        Machine Learning (ML) is a subset of artificial intelligence that enables computers to learn and make decisions from data without being explicitly programmed.
        
        Types of Machine Learning:
        
        1. Supervised Learning
        - Uses labeled training data
        - Examples: Classification, Regression
        - Algorithms: Linear Regression, Decision Trees, Random Forest
        
        2. Unsupervised Learning
        - Works with unlabeled data
        - Examples: Clustering, Dimensionality Reduction
        - Algorithms: K-Means, PCA, DBSCAN
        
        3. Reinforcement Learning
        - Learns through interaction with environment
        - Uses rewards and penalties
        - Examples: Game playing, Robotics
        
        Key Concepts:
        - Feature Engineering: Selecting and transforming input variables
        - Model Training: Teaching the algorithm using training data
        - Model Evaluation: Testing performance on unseen data
        - Overfitting: When model performs well on training but poorly on new data
        
        Popular Python Libraries:
        - Scikit-learn: General-purpose ML library
        - TensorFlow: Deep learning framework
        - PyTorch: Research-focused deep learning
        - Pandas: Data manipulation and analysis
        - NumPy: Numerical computing
        """,
        "metadata": {
            "category": "machine-learning",
            "difficulty": "beginner",
            "topics": ["supervised", "unsupervised", "reinforcement"]
        }
    }
]

# Ingest direct content documents
direct_response = make_request(
    "POST", 
    "/documents/ingest",
    data=direct_documents,
    params={
        "organization_id": ORGANIZATION_ID,
        "organization_name": ORGANIZATION_NAME
    }
)

print_response("Direct Content Ingestion", direct_response)

# Store document IDs for later use
direct_doc_ids = []
if "data" in direct_response and "document_ids" in direct_response["data"]:
    direct_doc_ids = direct_response["data"]["document_ids"]
    print(f"\n📝 Stored {len(direct_doc_ids)} document IDs from direct content")

In [ ]:
# Method 2: Simulated File Upload
print("\n📁 Method 2: Simulated File Upload")

# Simulate file content (in real scenario, this would be read from an uploaded file)
file_content = """
Web Development with Python and FastAPI

FastAPI is a modern, fast web framework for building APIs with Python 3.7+ based on standard Python type hints.

Key Features:
- Fast: Very high performance, on par with NodeJS and Go
- Fast to code: Increase the speed to develop features by about 200% to 300%
- Fewer bugs: Reduce about 40% of human (developer) induced errors
- Intuitive: Great editor support with completion everywhere
- Easy: Designed to be easy to use and learn
- Short: Minimize code duplication
- Robust: Get production-ready code with automatic interactive documentation

Getting Started:

1. Installation
```bash
pip install fastapi uvicorn
```

2. Basic Example
```python
from fastapi import FastAPI

app = FastAPI()

@app.get("/")
def read_root():
    return {"Hello": "World"}

@app.get("/items/{item_id}")
def read_item(item_id: int, q: str = None):
    return {"item_id": item_id, "q": q}
```

3. Run the server
```bash
uvicorn main:app --reload
```

Advanced Features:
- Automatic API documentation with Swagger UI
- Data validation using Pydantic models
- Dependency injection system
- Background tasks
- WebSocket support
- Authentication and authorization
- Database integration

FastAPI is perfect for building modern web APIs and microservices.
"""

# Create document from "file" content
file_documents = [{
    "title": "FastAPI Web Development Guide",
    "content": file_content,
    "metadata": {
        "source": "simulated_file_upload",
        "filename": "fastapi_guide.txt",
        "category": "web-development",
        "framework": "fastapi"
    }
}]

# Ingest file content
file_response = make_request(
    "POST", 
    "/documents/ingest",
    data=file_documents,
    params={"organization_id": ORGANIZATION_ID}
)

print_response("File Content Ingestion", file_response)

# Store document IDs
file_doc_ids = []
if "data" in file_response and "document_ids" in file_response["data"]:
    file_doc_ids = file_response["data"]["document_ids"]
    print(f"\n📝 Stored {len(file_doc_ids)} document IDs from file content")

In [ ]:
# Method 3: URL Scraping
print("\n🌐 Method 3: URL Scraping")

# Example URLs to scrape (replace with actual URLs you want to scrape)
url_requests = [
    {
        "url": "https://docs.python.org/3/tutorial/introduction.html",
        "title": "Python Tutorial Introduction",
        "organization_id": ORGANIZATION_ID,
        "scraping_method": "auto"
    }
]

url_doc_ids = []

for url_request in url_requests:
    print(f"\n🔍 Scraping: {url_request['url']}")
    
    url_response = make_request(
        "POST", 
        "/documents/ingest-url",
        data=url_request
    )
    
    print_response(f"URL Scraping: {url_request['title']}", url_response)
    
    if "data" in url_response and "document_id" in url_response["data"]:
        url_doc_ids.append(url_response["data"]["document_id"])

print(f"\n📝 Stored {len(url_doc_ids)} document IDs from URL scraping")

# Combine all document IDs
all_document_ids = direct_doc_ids + file_doc_ids + url_doc_ids
print(f"\n📚 Total documents ingested: {len(all_document_ids)}")
print(f"Document IDs: {all_document_ids}")

In [ ]:
# Verify organization and documents
org_docs_response = make_request("GET", f"/documents/organizations/{ORGANIZATION_ID}")
print_response("Organization Documents", org_docs_response)

if "data" in org_docs_response and "organization" in org_docs_response["data"]:
    org_data = org_docs_response["data"]["organization"]
    print(f"\n📊 Organization Summary:")
    print(f"   Name: {org_data['name']}")
    print(f"   Documents: {org_data['documents_count']}")
    print(f"   Created: {org_data['created_at']}")

## 4. Campaign Creation

In [ ]:
# Create a new campaign
campaign_data = {
    "name": "Python Learning Community Outreach 2024",
    "description": "Engage with Python learning communities to share knowledge and best practices",
    "response_tone": "helpful",
    "max_responses_per_day": 5
}

campaign_response = make_request(
    "POST", 
    "/campaigns/",
    data=campaign_data,
    params={"organization_id": ORGANIZATION_ID}
)

print_response("Campaign Creation", campaign_response)

# Store campaign ID
campaign_id = None
if "campaign" in campaign_response and campaign_response["campaign"]:
    campaign_id = campaign_response["campaign"]["id"]
    print(f"\n🎯 Campaign ID: {campaign_id}")
    print(f"📝 Campaign Name: {campaign_response['campaign']['name']}")
    print(f"📊 Status: {campaign_response['campaign']['status']}")
else:
    print("❌ Failed to create campaign")
    campaign_id = None

## 5. Topic Discovery

First, we'll extract relevant topics from our documents.

In [ ]:
if campaign_id and all_document_ids:
    print("🔍 Step 1: Discovering Topics from Documents")
    
    # Discover topics from selected documents
    topic_discovery_data = {
        "document_ids": all_document_ids[:3]  # Use first 3 documents for topic discovery
    }
    
    topic_response = make_request(
        "POST", 
        f"/campaigns/{campaign_id}/discover-topics",
        data=topic_discovery_data
    )
    
    print_response("Topic Discovery", topic_response)
    
    # Extract topics for next step
    discovered_topics = []
    if "data" in topic_response and "topics" in topic_response["data"]:
        discovered_topics = topic_response["data"]["topics"]
        print(f"\n📋 Discovered Topics:")
        for i, topic in enumerate(discovered_topics, 1):
            print(f"   {i}. {topic}")
    
    # Check campaign status
    status_response = make_request("GET", f"/campaigns/{campaign_id}/status")
    if "campaign" in status_response:
        print(f"\n📊 Campaign Status: {status_response['campaign']['status']}")
        print(f"📄 Documents Selected: {status_response['data']['documents_selected']}")
        
else:
    print("❌ Cannot proceed: Missing campaign ID or document IDs")
    discovered_topics = []

## 6. Subreddit Discovery

Now we'll use the discovered topics to find relevant subreddits.

In [ ]:
if campaign_id and discovered_topics:
    print("🎯 Step 2: Discovering Subreddits from Topics")
    
    # Discover subreddits based on topics
    subreddit_discovery_data = {
        "topics": discovered_topics
    }
    
    subreddit_response = make_request(
        "POST", 
        f"/campaigns/{campaign_id}/discover-subreddits",
        data=subreddit_discovery_data
    )
    
    print_response("Subreddit Discovery", subreddit_response)
    
    # Extract subreddits for next step
    target_subreddits = []
    if "data" in subreddit_response and "subreddits" in subreddit_response["data"]:
        target_subreddits = subreddit_response["data"]["subreddits"]
        print(f"\n🎯 Target Subreddits:")
        for i, subreddit in enumerate(target_subreddits, 1):
            print(f"   {i}. r/{subreddit}")
    
    # Check updated campaign status
    status_response = make_request("GET", f"/campaigns/{campaign_id}/status")
    if "campaign" in status_response:
        print(f"\n📊 Campaign Status: {status_response['campaign']['status']}")
        print(f"🎯 Subreddits Found: {status_response['data']['subreddits_found']}")
        
else:
    print("❌ Cannot proceed: Missing campaign ID or topics")
    target_subreddits = []

## 7. Post Discovery

In [ ]:
if campaign_id and target_subreddits:
    print("📝 Step 3: Discovering Relevant Posts")
    
    # Discover posts in target subreddits
    post_discovery_data = {
        "subreddits": target_subreddits[:3],  # Limit to first 3 subreddits for demo
        "max_posts_per_subreddit": 5,
        "time_filter": "week",
        "reddit_credentials": REDDIT_CREDENTIALS
    }
    
    posts_response = make_request(
        "POST", 
        f"/campaigns/{campaign_id}/discover-posts",
        data=post_discovery_data
    )
    
    print_response("Post Discovery", posts_response)
    
    # Extract post information
    target_posts = []
    if "data" in posts_response and "posts" in posts_response["data"]:
        target_posts = posts_response["data"]["posts"]
        print(f"\n📝 Found {len(target_posts)} relevant posts:")
        for i, post in enumerate(target_posts[:5], 1):  # Show first 5
            print(f"   {i}. r/{post['subreddit']}: {post['title'][:60]}...")
            print(f"      Relevance: {post['relevance_score']:.2f} - {post['relevance_reason']}")
    
    # Check campaign status
    status_response = make_request("GET", f"/campaigns/{campaign_id}/status")
    if "campaign" in status_response:
        print(f"\n📊 Campaign Status: {status_response['campaign']['status']}")
        print(f"📝 Posts Found: {status_response['data']['posts_found']}")
        
else:
    print("❌ Cannot proceed: Missing campaign ID or subreddits")
    target_posts = []

## 8. Response Generation

In [ ]:
if campaign_id and target_posts:
    print("💬 Step 4: Generating Responses")
    
    # Get post IDs for response generation
    post_ids = [post["id"] for post in target_posts[:3]]  # Limit to first 3 posts
    
    response_generation_data = {
        "target_post_ids": post_ids,
        "tone": "helpful"
    }
    
    generation_response = make_request(
        "POST", 
        f"/campaigns/{campaign_id}/generate-responses",
        data=response_generation_data
    )
    
    print_response("Response Generation", generation_response)
    
    # Show generated responses
    planned_responses = []
    if "data" in generation_response and "responses" in generation_response["data"]:
        planned_responses = generation_response["data"]["responses"]
        print(f"\n💬 Generated {len(planned_responses)} responses:")
        for i, response in enumerate(planned_responses, 1):
            print(f"\n   Response {i}:")
            print(f"   Target Post: {response['target_post_id']}")
            print(f"   Confidence: {response['confidence_score']:.2f}")
            print(f"   Content Preview: {response['response_content'][:100]}...")
    
    # Check campaign status
    status_response = make_request("GET", f"/campaigns/{campaign_id}/status")
    if "campaign" in status_response:
        print(f"\n📊 Campaign Status: {status_response['campaign']['status']}")
        print(f"💬 Responses Planned: {status_response['data']['responses_planned']}")
        
else:
    print("❌ Cannot proceed: Missing campaign ID or posts")
    planned_responses = []

## 9. Response Execution (Optional)

⚠️ **WARNING**: This step will actually post to Reddit if `ACTUALLY_POST_TO_REDDIT = True`

In [ ]:
if campaign_id and planned_responses:
    if ACTUALLY_POST_TO_REDDIT:
        print("🚀 Step 5: Executing Responses (POSTING TO REDDIT)")
        
        # Get response IDs for execution
        response_ids = [response["id"] for response in planned_responses[:2]]  # Limit to first 2
        
        execution_data = {
            "planned_response_ids": response_ids,
            "reddit_credentials": REDDIT_CREDENTIALS
        }
        
        execution_response = make_request(
            "POST", 
            f"/campaigns/{campaign_id}/execute-responses",
            data=execution_data
        )
        
        print_response("Response Execution", execution_response)
        
        # Show execution results
        if "data" in execution_response and "posted_responses" in execution_response["data"]:
            posted_responses = execution_response["data"]["posted_responses"]
            print(f"\n🚀 Execution Results:")
            for i, response in enumerate(posted_responses, 1):
                status = "✅ Success" if response["posting_successful"] else "❌ Failed"
                print(f"   Response {i}: {status}")
                if response["posting_successful"]:
                    print(f"   Reddit URL: https://reddit.com{response['reddit_permalink']}")
                else:
                    print(f"   Error: {response.get('error_message', 'Unknown error')}")
        
        # Final campaign status
        status_response = make_request("GET", f"/campaigns/{campaign_id}/status")
        if "campaign" in status_response:
            print(f"\n📊 Final Campaign Status: {status_response['campaign']['status']}")
            print(f"🚀 Responses Posted: {status_response['data']['responses_posted']}")
            print(f"✅ Successful Posts: {status_response['data']['successful_posts']}")
            print(f"❌ Failed Posts: {status_response['data']['failed_posts']}")
    else:
        print("⚠️  Step 5: Response Execution SKIPPED (Safe Mode)")
        print("\n🛡️  Reddit posting is disabled for safety.")
        print("   To enable posting, set ACTUALLY_POST_TO_REDDIT = True")
        print("   and provide valid Reddit credentials.")
        
        print(f"\n📋 Would have posted {len(planned_responses)} responses:")
        for i, response in enumerate(planned_responses, 1):
            print(f"   {i}. Response with confidence {response['confidence_score']:.2f}")
            print(f"      Content: {response['response_content'][:80]}...")
else:
    print("❌ Cannot proceed: Missing campaign ID or planned responses")

## 10. Analytics & Reporting

In [ ]:
print("📊 Step 6: Analytics & Reporting")

# Get organization quick stats
quick_stats_response = make_request("GET", f"/analytics/organizations/{ORGANIZATION_ID}/quick-stats")
print_response("Organization Quick Stats", quick_stats_response)

# Get organization performance report
performance_response = make_request("GET", f"/analytics/organizations/{ORGANIZATION_ID}/performance")
print_response("Organization Performance Report", performance_response)

if campaign_id:
    # Get campaign engagement report
    engagement_response = make_request("GET", f"/analytics/campaigns/{campaign_id}/engagement")
    print_response("Campaign Engagement Report", engagement_response)

# Get platform overview
platform_response = make_request("GET", "/analytics/platform/overview")
print_response("Platform Overview", platform_response)

## 11. Summary & Cleanup

In [ ]:
print("📋 Workflow Summary")
print("=" * 50)

print(f"🏢 Organization: {ORGANIZATION_NAME} ({ORGANIZATION_ID})")
print(f"📚 Documents Ingested: {len(all_document_ids)}")
print(f"   - Direct Content: {len(direct_doc_ids)}")
print(f"   - File Upload: {len(file_doc_ids)}")
print(f"   - URL Scraping: {len(url_doc_ids)}")

if campaign_id:
    print(f"\n🎯 Campaign: {campaign_id}")
    print(f"🔍 Topics Discovered: {len(discovered_topics)}")
    print(f"🎯 Subreddits Found: {len(target_subreddits)}")
    print(f"📝 Posts Analyzed: {len(target_posts)}")
    print(f"💬 Responses Generated: {len(planned_responses)}")
    
    if ACTUALLY_POST_TO_REDDIT:
        print(f"🚀 Responses Posted: Executed")
    else:
        print(f"🛡️  Responses Posted: Skipped (Safe Mode)")

print(f"\n⏰ Completed at: {datetime.now()}")
print(f"✅ Workflow completed successfully!")

# Display key IDs for reference
print("\n🔑 Key IDs for Reference:")
print(f"   Organization ID: {ORGANIZATION_ID}")
if campaign_id:
    print(f"   Campaign ID: {campaign_id}")
if all_document_ids:
    print(f"   Document IDs: {all_document_ids}")

print("\n" + "=" * 50)
print("🎉 Reddit Marketing AI Agent Workflow Complete!")

## Additional Operations

The following cells demonstrate additional operations you can perform:

In [ ]:
# Query documents
print("🔍 Document Query Example")

query_data = {
    "query": "machine learning algorithms",
    "organization_id": ORGANIZATION_ID,
    "method": "semantic",
    "top_k": 3
}

query_response = make_request("POST", "/documents/query", data=query_data)
print_response("Document Query", query_response)

if "documents" in query_response:
    print(f"\n📄 Found {len(query_response['documents'])} relevant documents:")
    for i, doc in enumerate(query_response["documents"], 1):
        print(f"   {i}. {doc['title']} (Score: {doc['score']:.3f})")
        print(f"      Content: {doc['content'][:100]}...")

In [ ]:
# List all campaigns for the organization
print("📋 List All Campaigns")

campaigns_response = make_request(
    "GET", 
    "/campaigns/",
    params={"organization_id": ORGANIZATION_ID}
)

print_response("Organization Campaigns", campaigns_response)

if "data" in campaigns_response and "campaigns" in campaigns_response["data"]:
    campaigns = campaigns_response["data"]["campaigns"]
    print(f"\n📊 Found {len(campaigns)} campaigns:")
    for i, campaign in enumerate(campaigns, 1):
        print(f"   {i}. {campaign['name']} ({campaign['status']})")
        print(f"      Created: {campaign['created_at']}")
        print(f"      ID: {campaign['id']}")

In [ ]:
# Test subreddit search
print("🔍 Subreddit Search Example")

search_response = make_request(
    "GET", 
    "/subreddits/search",
    params={"query": "python programming", "limit": 5}
)

print_response("Subreddit Search", search_response)

if "data" in search_response and "results" in search_response["data"]:
    results = search_response["data"]["results"]
    print(f"\n🎯 Found {len(results)} subreddits:")
    for i, subreddit in enumerate(results, 1):
        print(f"   {i}. r/{subreddit['name']} ({subreddit['subscribers']:,} subscribers)")
        print(f"      Description: {subreddit['description'][:80]}...")

## Conclusion

This notebook has demonstrated the complete workflow of the Reddit Marketing AI Agent:

1. ✅ **Setup & Configuration** - Validated environment and API connectivity
2. ✅ **Organization Setup** - Created/verified organization
3. ✅ **Document Ingestion** - Demonstrated all three ingestion methods
4. ✅ **Campaign Creation** - Created a marketing campaign
5. ✅ **Topic Discovery** - Extracted relevant topics from documents
6. ✅ **Subreddit Discovery** - Found relevant subreddits based on topics
7. ✅ **Post Discovery** - Identified relevant posts for engagement
8. ✅ **Response Generation** - Generated AI-powered responses
9. ✅ **Response Execution** - Demonstrated posting workflow (with safety controls)
10. ✅ **Analytics & Reporting** - Generated comprehensive reports

### Key Features Highlighted:
- **Multiple Document Sources**: Direct content, file upload, and URL scraping
- **AI-Powered Analysis**: Topic extraction, subreddit discovery, and response generation
- **Safety Controls**: Prevents accidental posting with configurable safety mode
- **Comprehensive Analytics**: Detailed reporting and performance tracking
- **Modular Workflow**: Each step can be run independently

### Next Steps:
1. **Configure Real Credentials**: Add your actual Reddit API credentials
2. **Enable Posting**: Set `ACTUALLY_POST_TO_REDDIT = True` when ready
3. **Customize Content**: Replace example documents with your actual content
4. **Monitor Performance**: Use the analytics endpoints to track campaign success
5. **Scale Operations**: Create multiple campaigns for different topics/audiences

The Reddit Marketing AI Agent is now ready for production use! 🚀